In [22]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
path='../input/churn-modellingcsv/Churn_Modelling.csv'
df=pd.read_csv(path,sep=",")

In [24]:
df.head()

In [25]:
X=df.iloc[:,3:13]
y=df.iloc[:,13]

In [26]:
X

In [27]:
y

In [28]:
# Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X["Gender"],drop_first=True)

In [29]:
# Concatenate Dataframes
X=pd.concat([X,geography,gender],axis=1)
X

In [30]:
# Drop unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)
X

In [31]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [32]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [39]:
## Hyperparameter optimisation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, BatchNormalization, Dropout
from tensorflow.keras.layers import LeakyReLU
from keras.activations import relu, sigmoid

In [43]:
def create_model(layers,activation):
    model=Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
model.add(Dense(units=1, kernel_initializer='glorot_uniform',activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
return model

model=KerasClassifier(build_fn=create_model,verbose=0)

layers=[(20,), (40,20), (45,30,15)]
activations=['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)
